In [1]:
from dotenv import load_dotenv
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = 'data/pdfs/Tese_Ficha_catalografica[v3].pdf' 
loader = PyPDFLoader(pdf_path)

docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory='./', collection_name='Tese')

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 6}
)

In [18]:
# Load database
vectorstore = Chroma(persist_directory='./', collection_name='Tese', embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
retriever.invoke('idh') 

'31 \n \nDe acordo com o Programa das Nações Unidas para o Desenvolvimento \n(PNUD) , o IDH, criado em 1990, consiste em um índice com variação de 0 a 1 , que \nmede de forma resumida o progresso do desenvolvimento humano a longo prazo a \npartir de três dimensões básicas: renda ; educação ; e saúde. Esse índice foi criado \ncom o objetivo de oferecer uma alternativa em relação a um indicador muito utilizado : \no Produto Interno Bruto (PIB)  per capita , utilizad o para medir o nível de \ndesenvolvimento , que considera va apenas a dimensão econô mica do \ndesenvolvimento. Assim, o IDH atua como uma medida que amplia a perspectiva \nsobre o desenvolvimento humano sem, contudo, esgotar todos os aspectos de \ndesenvolvimento. Os três pilares que constituem o IDH são mensurados da seguinte \nmaneira , de acord o com o PNUD:  \n \nUma vida longa e saudável (saúde) é medida pela expectativa de vida. \nO acesso ao conhecimento (educação) é medido por: i) média de anos\n\njovens de 11 a 13 a

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm

In [32]:
rag_chain.invoke('Qual o municipio que precisa de atenção urgente')

AIMessage(content='O município que precisa de atenção urgente é Porto Real, pois está associado ao círculo vicioso e apresenta indicadores essenciais de desenvolvimento humano abaixo da média.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 689, 'total_tokens': 718}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb7e6fb7-4c34-4f20-a0a2-c2f416efaee5-0', usage_metadata={'input_tokens': 689, 'output_tokens': 29, 'total_tokens': 718})

**Passando o resultado da consulta ao banco vetorial manualmente**

In [46]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | llm
    | StrOutputParser()
)

question = "Qual o municipio que precisa de atenção urgente?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'O município que precisa de atenção urgente é Porto Real, que está classificado na categoria de círculo vicioso e apresenta indicadores essenciais de desenvolvimento humano abaixo da média. Isso indica uma situação de vulnerabilidade que requer políticas públicas direcionadas.'